In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

In [2]:
from jubilee_controller import JubileeMotionController
import matplotlib.pyplot as plt
import time
import numpy as np 
from pynput import keyboard
import cv2
from datetime import datetime
from camera_controller import camera_tool
from email_sender import send_email
from image_processing import detect_circle

In [3]:
jubilee = JubileeMotionController()
jubilee.gcode('M98 P"/sys/config.g"')


'ok\n'

In [4]:
jubilee.home_all(mesh_mode_z=False)

In [8]:
jubilee.move_xyz_absolute(z=300)

In [6]:
camera = camera_tool(jubilee)

In [7]:
camera.install()

In [ ]:
posicao1 = [170, 200.0, 300.0]
posicao2 = [170.0, 327.0, 300.0]
posicoes = [posicao1,posicao2]

In [ ]:
n = 1000


for i in range(n):
    placa = ['placa 1','placa 2']
    last_saves = []
    covered_areas = []
    for posicao,p in zip(posicoes,placa):
        now = datetime.now()

        current_time_hms = now.strftime("%H:%M:%S")
        x,y,z = posicao
        jubilee.move_xyz_absolute(x=x,y=y,z=z,velocity=8000)
        time.sleep(2)
        save = f'fotos_experimento/{i} {p} {current_time_hms}.jpg'
        camera.photo(save,video_index=0)
        print(i,end='\r')  
        
        last_saves.append(save)

        coved_area1,coved_area2,no_grad_norm,masked = detect_circle(save, (100,210),(77,180),plot_image=False,save_name=f'atual_analise {p}.jpg')
        covered_areas.append((coved_area1,coved_area2))
        last_saves.append(f'atual_analise {p}.jpg')

    
    try:
        send_email("Experimento Análise de Meio de Cultura",f"Iteração {i}: Área coberta = {covered_areas} ",last_saves)
        pass

    except:
        continue
    
    time.sleep(1800)